## Callin Switzer
April 2019

Multiprocessing with multiple arguments

In [29]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import seaborn as sns
from scipy.integrate import odeint
import random
import time
from datetime import datetime
import sys
from multiprocessing import Pool, cpu_count
import simUtils # note that this is a custom-written file 
import importlib
import functools

print(sys.version)

3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]


In [30]:
now = datetime.now()
print("last run on " + str(now))


last run on 2019-05-01 07:01:11.748459


In [31]:
np.random.seed(123)

In [32]:
from collections import OrderedDict

globalDict = OrderedDict({"bhead": 0.507,
            "ahead": 0.908,
            "bbutt": 0.1295,
            "abutt": 1.7475, 
            "rho": 1, 
            "rhoA": 0.00118, 
            "muA": 0.000186, 
            "L1": 0.908, 
            "L2": 1.7475,  
            "L3": 0.75,
            "K": 29.3,
            "c":  14075.8,
            "g": 980.0,
            "betaR":  0.0,
            "nstep": 100,
            "nrun" : 2500  # (max) number of  trajectories.
            })

In [33]:
# Calculated variables
globalDict['m1'] = globalDict['rho']*(4/3)*np.pi*(globalDict['bhead']**2)*globalDict['ahead']
globalDict["m2"] = globalDict["rho"]*(4/3)*np.pi*(globalDict["bbutt"]**2)*globalDict["abutt"]
globalDict["echead"] = globalDict["ahead"]/globalDict["bhead"]
globalDict['ecbutt'] = globalDict['abutt']/globalDict['bbutt']
globalDict['I1'] = (1/5)*globalDict['m1']*(globalDict['bhead']**2)*(1 + globalDict['echead']**2)
globalDict['I2'] = (1/5)*globalDict['m2']*(globalDict['bbutt']**2)*(1 + globalDict['ecbutt']**2)
globalDict['S_head'] = np.pi*globalDict['bhead']**2
globalDict['S_butt'] = np.pi*globalDict['bbutt'] **2
t = np.linspace(0, 0.02, num = globalDict["nstep"], endpoint = True)

In [34]:
# ranges for variables
rangeDict = {"Fmin": 0,
             "Fmax": 44300,
            "alphaMin":  0,
             "alphaMax":2*np.pi, 
            "tau0Min": -100000, 
             "tau0Max": 100000}

In [35]:
# ranges for initial conditions
ranges = np.array([[rangeDict["Fmin"], rangeDict["Fmax"]], 
                   [rangeDict["alphaMin"], rangeDict["alphaMax"]], 
                   [rangeDict["tau0Min"], rangeDict["tau0Max"] ]])


In [36]:
# generate initial conditions for state 0
# x,xd,y,yd,theta,thetad,phi,phid
state0_ICs = [0.0, 0.0001, 0.0, 0.0001, 0.0, 0.0001, 0.0, 0.0001]

# F, alpha, tau
FAlphaTau_list = np.random.uniform(ranges[:, 0], ranges[:, 1], 
                                   size=(globalDict["nrun"], ranges.shape[0]))

In [37]:
# convert dict to list, since @jit works better with lists
globalList = [ v for v in globalDict.values() ]

In [38]:
# try to run simulations in parallel
p = Pool(cpu_count() - 1)
stt = time.time()
bb = p.map(functools.partial(simUtils.flyBug_dictInput, t=t, 
                              state0_ICs = state0_ICs, 
                              FAlphaTau_list= FAlphaTau_list, 
                              globalList = globalList), range(globalDict["nrun"]))
print(time.time() - stt)
p.close()
p.join()

3.4507710933685303


In [39]:
np.array(bb).shape # shape is nrun, num conditions, timesteps

(2500, 8, 100)

In [40]:
# this shows just the first run
pd.DataFrame(bb[0], index = ["x", "xd","y","yd","theta","thetad","phi","phid"])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
x,0.0000,-0.000128,-0.000511,-0.001142,-0.002011,-0.003107,-0.004414,-0.005914,-0.007584,-0.009400,...,2.971913,2.984507,2.991001,2.991335,2.985495,2.973517,2.955491,2.931565,2.901941,2.866883
xd,0.0001,-1.268292,-2.514038,-3.721797,-4.876143,-5.961639,-6.962836,-7.864279,-8.650505,-9.306052,...,77.279419,47.311974,16.930700,-13.638267,-44.152657,-74.355179,-103.975190,-132.730549,-160.329775,-186.474534
y,0.0000,0.001281,0.004519,0.009683,0.016773,0.025791,0.036738,0.049614,0.064421,0.081159,...,3.096095,3.060931,3.026215,2.992484,2.960292,2.930208,2.902810,2.878683,2.858408,2.842564
yd,0.0001,11.261442,20.793850,30.329312,39.868630,49.412386,58.960904,68.514197,78.071909,87.633254,...,-174.313145,-173.387773,-169.857734,-163.624390,-154.607464,-142.747220,-128.006814,-110.374478,-89.865597,-66.524639
theta,0.0000,-0.000879,-0.002651,-0.005273,-0.008744,-0.013064,-0.018233,-0.024252,-0.031120,-0.038838,...,-3.484899,-3.562292,-3.640536,-3.719632,-3.799579,-3.880378,-3.962028,-4.044530,-4.127882,-4.212086
thetad,0.0001,-6.673432,-10.875968,-15.078930,-19.282316,-23.486089,-27.690213,-31.894653,-36.099372,-40.304337,...,-380.987851,-385.202373,-389.416747,-393.631207,-397.845862,-402.060583,-406.275139,-410.489220,-414.702468,-418.914516
phi,0.0000,-0.000177,-0.001178,-0.003028,-0.005727,-0.009276,-0.013674,-0.018921,-0.025017,-0.031963,...,-3.415008,-3.491605,-3.569052,-3.647350,-3.726498,-3.806497,-3.887347,-3.969047,-4.051597,-4.134997
phid,0.0001,-2.854390,-7.056899,-11.259892,-15.463308,-19.667109,-23.871258,-28.075717,-32.280451,-36.485424,...,-377.048731,-381.259107,-385.469227,-389.679324,-393.889516,-398.099687,-402.309608,-406.518973,-410.727423,-414.934591


In [ ]:
# refref: now, save data to database or .csv file